In [ ]:
%load_ext kedro.ipython

In [2]:
import polars as pl 
import numpy as np

pl.Config(tbl_rows=50)

In [ ]:
simulating_alternative_openers = catalog.load("simulating_alternative_openers")

In [4]:
simulating_alternative_openers.head(12)

index,challenge,guess,match_pattern,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
u32,str,str,str,str,str,u8,u8,u8,u16,str,bool,str,u32,u8,str,str,u8
0,"""yummy""","""soare""","""BBBBB""","""saeor""","""""",5,0,0,538,"""thing, child, pupil, cubic, im…",false,"""linty""",0,null,"""very hard""","""soare""",1
1,"""yummy""","""linty""","""BBBBG""","""nisaeotlr""","""y""",9,1,1,50,"""fuzzy, mummy, puffy, pudgy, du…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1
2,"""yummy""","""soare""","""BBBBB""","""saeor""","""""",5,0,0,538,"""think, fight, click, quint, fi…",false,"""milty""",1,null,"""very hard""","""soare""",1
3,"""yummy""","""milty""","""YBBBG""","""isaeotlr""","""my""",8,2,1,10,"""dummy, bumpy, pygmy, gummy, ju…",true,"""dummy""",1,null,"""very hard""","""milty""",1
4,"""yummy""","""soare""","""BBBBB""","""saeor""","""""",5,0,0,538,"""thing, civil, input, thick, ni…",false,"""minty""",2,null,"""very hard""","""soare""",1
5,"""yummy""","""minty""","""YBBBG""","""nisaeotr""","""my""",8,2,1,13,"""dummy, lumpy, bumpy, pygmy, gu…",true,"""dummy""",2,null,"""very hard""","""minty""",1
6,"""yummy""","""soare""","""BBBBB""","""saeor""","""""",5,0,0,538,"""nicht, night, fifth, ninth, ly…",false,"""clint""",3,null,"""very hard""","""soare""",1
7,"""yummy""","""clint""","""BBBBB""","""ncisaeotlr""","""""",10,0,0,48,"""fuzzy, puppy, mummy, yummy, pu…",true,"""fuzzy""",3,null,"""very hard""","""clint""",1
8,"""yummy""","""soare""","""BBBBB""","""saeor""","""""",5,0,0,538,"""blind, light, civic, unity, th…",false,"""unlit""",4,null,"""very hard""","""soare""",1


In [5]:
simulating_alternative_openers.height

4544000

## Retrieve the 2nd row of every group

### Determine the minimum number of choices remaining after the second guess. This will give us an idea of the best second guess.

In [6]:
second_guess_df_2 = simulating_alternative_openers.filter(pl.col("guess") != "soare")
second_guess_df_2.head()

index,challenge,guess,match_pattern,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
u32,str,str,str,str,str,u8,u8,u8,u16,str,bool,str,u32,u8,str,str,u8
1,"""yummy""","""linty""","""BBBBG""","""nisaeotlr""","""y""",9,1,1,50,"""fuzzy, mummy, puffy, pudgy, du…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1
3,"""yummy""","""milty""","""YBBBG""","""isaeotlr""","""my""",8,2,1,10,"""dummy, bumpy, pygmy, gummy, ju…",true,"""dummy""",1,null,"""very hard""","""milty""",1
5,"""yummy""","""minty""","""YBBBG""","""nisaeotr""","""my""",8,2,1,13,"""dummy, lumpy, bumpy, pygmy, gu…",true,"""dummy""",2,null,"""very hard""","""minty""",1
7,"""yummy""","""clint""","""BBBBB""","""ncisaeotlr""","""""",10,0,0,48,"""fuzzy, puppy, mummy, yummy, pu…",true,"""fuzzy""",3,null,"""very hard""","""clint""",1
9,"""yummy""","""unlit""","""YBBBB""","""nisaeotlr""","""u""",9,1,0,60,"""chuck, fuzzy, duchy, puppy, bu…",false,"""chuck""",4,null,"""very hard""","""unlit""",1


In [7]:
second_guess_df_2.height

2271750

In [8]:
best_guess = (
    second_guess_df_2
    .group_by("guess")
    .agg(avg_min_choice_after_2nd_guess = pl.col("num_choices_after_guess").mean().cast(pl.Float32))
    .sort("avg_min_choice_after_2nd_guess", descending=False)
)

In [9]:
best_guess.head(10)

guess,avg_min_choice_after_2nd_guess
str,f32
"""linty""",16.219435
"""clint""",17.165401
"""unlit""",17.4874
"""milty""",17.751513
"""glint""",18.555738
"""minty""",18.875317
"""lytic""",19.27919
"""tulip""",19.598768
"""lindy""",19.886652


In [10]:
best_singles = catalog.load("best_singles")

[07/19/24 10:33:21] INFO     Loading data from best_singles (EagerPolarsDataset)...             ]8;id=620876;file:///Users/jeraime/opt/miniconda3/envs/wordle/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=694824;file:///Users/jeraime/opt/miniconda3/envs/wordle/lib/python3.10/site-packages/kedro/io/data_catalog.py#508\508]8;;\

In [11]:
best_single = (
    best_singles
    ["word"]
    .head(1)
    .item()
)

best_single

'soare'

In [12]:
best_alt_second_guess = (
    second_guess_df_2
    .group_by("guess")
    .agg(avg_min_choice_after_2nd_guess = pl.col("num_choices_after_guess").mean().cast(pl.Float32))
    .sort("avg_min_choice_after_2nd_guess", descending=False)
    ["guess"]
    .head(1)
    .item()
)

best_alt_second_guess

'linty'

In [13]:
openers = best_single, best_alt_second_guess

In [14]:
openers

('soare', 'linty')

In [15]:
openers[0]

'soare'